In [1]:
import requests 
from bs4 import BeautifulSoup as bs 
import pandas as pd

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/51.0.2704.63 Safari/537.36'}
r=requests.get('https://cryptoslam.io/', headers=headers)
content = bs(r.content)

#print(content.prettify())

In [5]:
table=content.select('table.table-hover.js-top-by-sales-table-all.summary-sales-table')[0]
columns = table.find("thead").find_all("th")
column_names = [c.string for c in columns]

table_rows = table.find('tbody').find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [str(tr.get_text()).strip() for tr in td]
    l.append(row)

q = []
for tr in table_rows:
    td = tr.select('td span.summary-sales-table__column-product-name')
    row = [str(tr.get_text()).strip() for tr in td]
    q.append(row)


df1 = pd.DataFrame(q, columns=['Collections'])
df1 = df1.loc[~df1.index.duplicated(keep='first')]
df2 = pd.DataFrame(l, columns=column_names)
df2 = df2.loc[~df2.index.duplicated(keep='first')]

nft_all_time = df1.join(df2)
nft_all_time.drop(nft_all_time.columns[[2,3]],axis=1,inplace=True)
nft_all_time.head(250)

,Collections,Sales,Buyers,Txns,Owners
0,Axie Infinity,"$4,046,809,279","1,687,642","15,347,106","2,655,681"
1,CryptoPunks,"$2,167,147,093","5,390","20,673","3,537"
2,Bored Ape Yacht Club,"$1,726,215,824","10,609","27,677",
3,Art Blocks,"$1,221,783,573","27,999","149,477","32,853"
4,Mutant Ape Yacht Club,"$1,117,853,515","18,473","33,468",
...,...,...,...,...,...
245,BrainDrops,"$14,247,639","2,381","6,886","2,360"
246,Bulls On The Block,"$14,153,457","6,859","19,833","4,850"
247,uwucrew,"$14,097,376","7,094","15,840",
248,Alpha Kongs Club,"$14,034,226","7,638","12,538",


In [4]:
nft_all_time.to_csv('nft_sales.csv',index=False)